In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "neo4j+s://2bb967b7.databases.neo4j.io:7687", 
    auth=basic_auth("neo4j", "R4t4Mta4Z-qidT1aY6UNWUH701RJD1oAFfubc-gWJG8"))
session = driver.session()

In [93]:
cypher_query = '''
MATCH (a)
RETURN a
'''

results = session.run(cypher_query,
  parameters={})

In [144]:
for record in results:
    print(record)

## 0. Открываем файлы

In [10]:
import json

In [74]:
with open('names.txt', 'r', encoding='utf-8') as f:
    names = f.read()
names = names.split('\n')

In [103]:
with open('cities.txt', 'r', encoding='utf-8') as f:
    cities = f.read()
cities = cities.split('\n')

In [120]:
cities

['Царичанка, Украина',
 'Беллвилл, США',
 'Чечон, Южная Корея',
 'Макушино, Россия',
 'Утта, Россия',
 'Карели, Грузия',
 'Вердан, Канада',
 'Софрино, Россия',
 'Вест-Варехам, США',
 'Кочево, Россия',
 'Вахтан, Россия',
 'Титагарх, Индия',
 'Норт-Белмонт, США',
 'Тируччираппалли, Индия',
 'Темпл, США',
 'Дормидонтовка, Россия',
 'Виго, Испания',
 'Вуктыл, Россия',
 'Гаруа, Камерун',
 'Юргамыш, Россия']

In [121]:
countries_dict = {}
for c in cities:
    city = c.split(',')[0]
    country = c.split(',')[1].strip()
    if country not in countries_dict.keys():
        countries_dict[country] = []
    countries_dict[country].append(city)

In [122]:
countries_dict 

{'Украина': ['Царичанка'],
 'США': ['Беллвилл', 'Вест-Варехам', 'Норт-Белмонт', 'Темпл'],
 'Южная Корея': ['Чечон'],
 'Россия': ['Макушино',
  'Утта',
  'Софрино',
  'Кочево',
  'Вахтан',
  'Дормидонтовка',
  'Вуктыл',
  'Юргамыш'],
 'Грузия': ['Карели'],
 'Канада': ['Вердан'],
 'Индия': ['Титагарх', 'Тируччираппалли'],
 'Испания': ['Виго'],
 'Камерун': ['Гаруа']}

In [ ]:
cities_dict = {c.split(',')[1]: for c in cities}

In [15]:
with open('interests.txt', 'r', encoding='utf-8') as f:
    interests = f.read()
interests = interests.split('\n')

In [11]:
with open('books_info.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

## 1. Люди и связи между ними

### 1.1 Nodes. Люди

In [38]:
import random as rnd

In [43]:
import tqdm as tqdm 

In [40]:
create_p_q = []
for name in names:
    age = rnd.randint(18, 81)
    create_people_q = """CREATE (p:Person {name:'%s', age:'%s'})""" % (name, age)
    create_p_q.append(create_people_q)

In [41]:
create_p_q

["CREATE (p:Person {name:'Константинов Алексей', age:'57'})",
 "CREATE (p:Person {name:'Данилов Владислав', age:'59'})",
 "CREATE (p:Person {name:'Харитонов Артём', age:'51'})",
 "CREATE (p:Person {name:'Зайцев Иван', age:'75'})",
 "CREATE (p:Person {name:'Пантелеева Александра', age:'35'})",
 "CREATE (p:Person {name:'Васильева Агния', age:'58'})",
 "CREATE (p:Person {name:'Чистякова Валерия', age:'31'})",
 "CREATE (p:Person {name:'Богданов Илья', age:'24'})",
 "CREATE (p:Person {name:'Макаров Кирилл', age:'50'})",
 "CREATE (p:Person {name:'Антонов Пётр', age:'29'})",
 "CREATE (p:Person {name:'Иванов Руслан', age:'67'})",
 "CREATE (p:Person {name:'Капустина Валерия', age:'78'})",
 "CREATE (p:Person {name:'Еремина Милана', age:'32'})",
 "CREATE (p:Person {name:'Петухов Владимир', age:'54'})",
 "CREATE (p:Person {name:'Киселева Варвара', age:'53'})",
 "CREATE (p:Person {name:'Соколов Михаил', age:'47'})",
 "CREATE (p:Person {name:'Казаков Георгий', age:'29'})",
 "CREATE (p:Person {name:'

In [42]:
for q in tqdm(create_p_q):
    _ = session.run(q)

### 1.2 Вершины - страны, ребра - города

In [114]:
countries = set(city.split(',')[1].strip() for city in cities)

In [115]:
countries

{'Грузия',
 'Индия',
 'Испания',
 'Камерун',
 'Канада',
 'Россия',
 'США',
 'Украина',
 'Южная Корея'}

In [133]:
create_c_qs = []
for c in countries:
    create_c_q = """CREATE (c:Country {name:'%s'})""" % (c)
    create_c_qs.append(create_c_q)

In [134]:
create_c_qs

["CREATE (c:Country {name:'США'})",
 "CREATE (c:Country {name:'Камерун'})",
 "CREATE (c:Country {name:'Грузия'})",
 "CREATE (c:Country {name:'Россия'})",
 "CREATE (c:Country {name:'Испания'})",
 "CREATE (c:Country {name:'Индия'})",
 "CREATE (c:Country {name:'Южная Корея'})",
 "CREATE (c:Country {name:'Канада'})",
 "CREATE (c:Country {name:'Украина'})"]

In [135]:
for q in tqdm(create_c_qs):
    _ = session.run(q)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.91it/s]


### 1.3 Вершины - интересы

In [100]:
create_int_qs = []
for interest in interests:
    create_int_q = """CREATE (c:Interest {name:'%s'})""" % (interest)
    create_int_qs.append(create_int_q)

In [102]:
for q in tqdm(create_int_qs):
    _ = session.run(q)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.97it/s]


In [101]:
create_int_qs

["CREATE (c:Interest {name:'Детективы'})",
 "CREATE (c:Interest {name:'Фэнтези'})",
 "CREATE (c:Interest {name:'Русская литература'})",
 "CREATE (c:Interest {name:'Зарубежная литература'})",
 "CREATE (c:Interest {name:'Фантастика'})",
 "CREATE (c:Interest {name:'Психология'})",
 "CREATE (c:Interest {name:'Политика'})",
 "CREATE (c:Interest {name:'Ужасы'})",
 "CREATE (c:Interest {name:'Романы'})",
 "CREATE (c:Interest {name:'Кино'})"]

### 2.1 Добавляем ребра - книги

In [56]:
import datetime

In [65]:
def gen_dates(n):
    dates = []
    for _ in range(n):
        startdate=datetime.date(2021,12,1)
        date=startdate+datetime.timedelta(rnd.randint(1,101))
        dates.append(date.strftime("%Y-%m-%d"))
    return dates

In [66]:
dates = gen_dates(70)

In [78]:
send_book_qs = []
for _ in range(70):
    names_q = rnd.choices(names, k=2)
    if names_q[0] == names_q[1]:
        continue
    date = rnd.choice(dates)
    book_name = rnd.choice(list(data.keys()))
    author = data[book_name]['author']
    genre = data[book_name]['genre']
    pub = data[book_name]['pub_year']
    
    
    
    send_books = """
        MATCH (a:Person {name:'%s'}) 
        MATCH (b:Person {name:'%s'})
        Create (a)-[:SENT_BOOK_TO {date: "%s", book_name: "%s",
                    author: "%s", genre : "%s", bup_year : "%s"}]->(b)
        return *
        """ % (names_q[0], names_q[1], date, book_name, author, genre, pub)
    
    send_book_qs.append(send_books)

In [82]:
from tqdm import tqdm

In [88]:
for q in tqdm(send_book_qs[35:]):
    _ = session.run(q)

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [00:10<00:00,  3.11it/s]


In [87]:
send_book_qs[35:]

['\n        MATCH (a:Person {name:\'Кравцова Анастасия\'}) \n        MATCH (b:Person {name:\'Макаров Кирилл\'})\n        Create (a)-[:SENT_BOOK_TO {date: "2022-01-27", book_name: "Иметь или быть?",\n                    author: "Эрих Фромм", genre : "Классическая психология", bup_year : "2022"}]->(b)\n        return *\n        ',
 '\n        MATCH (a:Person {name:\'Моисеева Елена\'}) \n        MATCH (b:Person {name:\'Казакова Алиса\'})\n        Create (a)-[:SENT_BOOK_TO {date: "2022-03-03", book_name: "Тайная история",\n                    author: "Донна Тартт", genre : "Современная зарубежная проза", bup_year : "2022"}]->(b)\n        return *\n        ',
 '\n        MATCH (a:Person {name:\'Данилов Владислав\'}) \n        MATCH (b:Person {name:\'Петухов Владимир\'})\n        Create (a)-[:SENT_BOOK_TO {date: "2022-02-06", book_name: "Как ты умрешь",\n                    author: "Майк Омер", genre : "Триллеры", bup_year : "2022"}]->(b)\n        return *\n        ',
 '\n        MATCH (a:Pe

### 2.2 Ребра человек-город->страна

In [132]:
rnd.choice(list(countries_dict['Россия']))

'Дормидонтовка'

In [138]:
lives_in_qs = []
for name in names:
    country = rnd.choice(list(countries_dict.keys()))
    city = rnd.choice(countries_dict[country])
    
    lives_in = """
        MATCH (a:Person {name:'%s'}) 
        MATCH (c:Country {name:'%s'})
        Create (a)-[:LIVES_IN {city : '%s'}]->(c)
        return *
        """ % (name, country, city)
    
    lives_in_qs.append(lives_in)

In [140]:
for q in tqdm(lives_in_qs):
    _ = session.run(q)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:15<00:00,  3.14it/s]


### 2.3 Ребра человек->интерес

In [141]:
int_in_qs = []

for name in names[:35]:
    interest = rnd.choice(interests)
    
    int_in = """
        MATCH (a:Person {name:'%s'}) 
        MATCH (c:Interest {name:'%s'})
        Create (a)-[:INTERESTED_IN]->(c)
        return *
        """ % (name, interest)
    
    int_in_qs.append(int_in)

In [142]:
int_in_qs

["\n        MATCH (a:Person {name:'Константинов Алексей'}) \n        MATCH (c:Interest {name:'Зарубежная литература'})\n        Create (a)-[:INTERESTED_IN]->(c)\n        return *\n        ",
 "\n        MATCH (a:Person {name:'Данилов Владислав'}) \n        MATCH (c:Interest {name:'Фэнтези'})\n        Create (a)-[:INTERESTED_IN]->(c)\n        return *\n        ",
 "\n        MATCH (a:Person {name:'Харитонов Артём'}) \n        MATCH (c:Interest {name:'Русская литература'})\n        Create (a)-[:INTERESTED_IN]->(c)\n        return *\n        ",
 "\n        MATCH (a:Person {name:'Зайцев Иван'}) \n        MATCH (c:Interest {name:'Психология'})\n        Create (a)-[:INTERESTED_IN]->(c)\n        return *\n        ",
 "\n        MATCH (a:Person {name:'Пантелеева Александра'}) \n        MATCH (c:Interest {name:'Русская литература'})\n        Create (a)-[:INTERESTED_IN]->(c)\n        return *\n        ",
 "\n        MATCH (a:Person {name:'Васильева Агния'}) \n        MATCH (c:Interest {name:'Зар

In [143]:
for q in tqdm(int_in_qs):
    _ = session.run(q)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:11<00:00,  3.00it/s]


In [ ]:
cypher_query = """
MATCH (a:Person {name:'Jane'}) 
MATCH (b:Person {name:'Jack'})
Create (a)-[:SENT_BOOK_TO {date: '{}', book_name: '{}',
            author: '{}', genre : '{}', bup_date : '{}'}]->(b)
return *
""".format()